In [1]:
from pathlib import Path
import sys
import pandas as pd
import numpy as np
import pickle
import sklearn
from datetime import datetime
from sklearn.tree import DecisionTreeClassifier
from modAL.models import ActiveLearner

package_path = Path.cwd().parent
if package_path not in sys.path:
    sys.path.append(str(package_path))

from utils.dataset_class import Phase3DataSet, Setting
from custom_pipeline import get_model_pkl, dump_model_pkl
from models.meta.platipus_class import Platipus

ModuleNotFoundError: No module named 'torch'